# Libraries

In [1]:
import matplotlib
import json

matplotlib.use("TkAgg")
import gym
import gridworld
from gym import wrappers, logger
import numpy as np
import copy
import re 

# np.set_printoptions(linewidth=40)

np.random.seed(3)

# Understanding and Exploring openAi Framework

In [2]:
import ast

### create the env
env = gym.make("gridworld-v0")

# Init the seed  ( pseudo random )
env.seed(0) 

print("-----------------------------------------------  Introduction  ------------------------------------------ ")

# Print possible actions
print("possible actions : ")  
print("----------------")
print(env.action_space)

# do action 1
# return : Observation(next state) , reward , done ( Boolean : next state is terminal state so the game is finished)
print("\n after doing action 1 we get : ")
print(env.step(0))  
print("----------------")
# Visualize the game ( the grid ) 
# env.render()  
# Visualize the grid game in a console mode
# env.render(mode="human")

# get statedic , MDP
statedic, mdp = env.getMDP()
# statedic : state -> encoded_number 
print("\n all possible states (statedic) :")
print("----------------")
for state in statedic : 
    l = ast.literal_eval(state)
    print(np.array(l))
    print("==> ",statedic[state])
    print("----------------")
print("number of States : ",len(statedic))
print("----------------")
print("-------------------------------------  Example of a state and its transitions  ------------------------------ ")
#         MDP : dict ( state => transitions from this state )
# transitions : dict { action => [ (proba1 , next_state1 , reward1 , done) , (proba2 , next_state2 , reward2 , done) ... ] }
state, transitions = list(mdp.items())[0]  # state 0 for example  
print(" state : " )
print(np.array(ast.literal_eval(state)))
print("----------------")
print(" \n\n its transitions : ")
print("----------------\n\n")
for a in transitions :
    print("##########")
    print("action ",a)
    print("##########")
    for nuplet in transitions[a] : 
        print("next state : " )
        print(np.array(ast.literal_eval(nuplet[1])))
        print("Proba  : " , nuplet[0] )
        print("Reward : " , nuplet[2] )
        print("Done   : " , nuplet[3] )
        print("----------------")

-----------------------------------------------  Introduction  ------------------------------------------ 
possible actions : 
----------------
Discrete(4)

 after doing action 1 we get : 
(array([[1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 3, 1],
       [1, 0, 1, 0, 5, 1],
       [1, 0, 0, 0, 2, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1]]), 0, False, {})
----------------

 all possible states (statedic) :
----------------
[[1 1 1 1 1 1]
 [1 0 0 0 3 1]
 [1 0 1 0 5 1]
 [1 0 0 0 2 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  0
----------------
[[1 1 1 1 1 1]
 [1 0 0 0 3 1]
 [1 0 1 0 2 1]
 [1 0 0 0 0 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  1
----------------
[[1 1 1 1 1 1]
 [1 0 0 0 3 1]
 [1 0 1 0 5 1]
 [1 0 0 2 0 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  2
----------------
[[1 1 1 1 1 1]
 [1 0 0 0 3 1]
 [1 0 1 2 5 1]
 [1 0 0 0 0 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  3
----------------
[[1 1 1 1 1 1]
 [1 0 0 2 3 1]
 [1 0 1 0 5 1]
 [1 0 0 0 0 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  4
-----

**Terminal States are not on the MDP**

In [3]:
missingMDP = set(statedic.keys()).difference(set(env.getMDP()[1].keys()))
print("missing from MDP ")
for s in missingMDP : 
    l = ast.literal_eval(s)
    print(np.array(l))
    print("==> ",statedic[s])

missing from MDP 
[[1 1 1 1 1 1]
 [1 0 0 0 3 1]
 [1 0 1 0 2 1]
 [1 0 0 0 0 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  1
[[1 1 1 1 1 1]
 [1 0 0 0 2 1]
 [1 0 1 0 5 1]
 [1 0 0 0 0 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
==>  5


**they are encoded :**
- **Win (Green) : 5**
- **Defeat (Red) : 1**

Grid World Goal : 

The Agent **Blue** should harvest **Yellow** elements  and finish on a **Green** case .

The agent should avoid **Pink** cases which penalizes , and **Red** ones which are terminal states  .

**Encoding Mapping** : 

0 => Empty Case 

1 => Wall

2 => Player ( Our Agent )

3 => Green Case

4 => Yellow Case

5 => Red Case

6 => Pink Case

We have different plans  **0 ===> 10** with several difficulty settings

In [4]:
statedic , MDP = env.getMDP()

S = len(statedic)
A = env.action_space.n

In [5]:
def gen_policy ( S , A , init="Deterministic" ) : 
    
    '''
    S : number of states 
    A : number of actions
    init : ( 'Deterministic' | 'Random' | 'Uniform' )
    '''
    if ( init == 'Uniform' ) :
        return np.ones((S,A),dtype=np.float16) / A
    
    p = np.zeros((S,A),dtype=np.float16)
    
    if ( init == 'Deterministic' ) : 
        p[ list(range(9)) , np.random.choice ( 4 , size=9 ) ] = 1
        return p 
        
    # else random  rounded 1e-1  like [ 0.6 , 0.2 , 0.0 , 0.1 ]
    for i in range (S) :
        Sum = 0 
        for j in range (A) :
            if ( j < A-1 ) : 
                p[i][j] = round( np.random.rand() * ( 1-Sum ) , 1 )
                Sum += p[i][j]
                if ( Sum >= 1 ):
                    break
            else :
                p[i][j] = (1-Sum)
    return np.round(p,1)

In [6]:
deter_policy =  gen_policy ( S , A )
rand_policy  =  gen_policy ( S , A , 'Random' )
uni_policy   =  gen_policy ( S , A , 'Uniform')

In [7]:
deter_policy

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float16)

In [8]:
rand_policy

array([[0. , 0. , 0.2, 0.8],
       [0.1, 0.6, 0. , 0.3],
       [0.5, 0.1, 0.1, 0.3],
       [0.5, 0.5, 0. , 0. ],
       [0.8, 0.2, 0. , 0. ],
       [0.6, 0.1, 0.2, 0.1],
       [0.3, 0.3, 0.3, 0.1],
       [0.8, 0.1, 0.1, 0. ],
       [0.7, 0. , 0. , 0.3],
       [0.3, 0.6, 0. , 0.1],
       [0.4, 0.2, 0.3, 0.1]], dtype=float16)

In [9]:
uni_policy

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]], dtype=float16)

# Algorithms

In [10]:
def Policy_eval ( env , Policy , Eps=1e-5 , discount=0.99 , verbose =True ) :
    
    # init 
    statedic , MDP = env.getMDP()
    V_prev = np.random.rand( env.nS )
    actions = np.arange( env.nA )
    
    #for printing 
    n_iter_eval = 1
    
    # loop applying Bellman Operator untill convergence   
    while(True) : 
        V_curr = np.zeros ( V_prev.shape ) 
        # for each state s_t
        for s in MDP :
            Sum_a = 0 
            # action according to policy
            for a in MDP[s] :
                Pi_s_a = Policy[ statedic[s] ][a]
                if ( Pi_s_a > 0 ) :
                    # for each state s_t+1 
                    Sum_snext = 0
                    for proba, next_state, reward, done in MDP[s][a] :
                        Sum_snext +=   (proba * ( reward + discount * V_prev[ statedic[next_state] ] ))
                    
                    Sum_a += Pi_s_a * Sum_snext
                    
            V_curr[ statedic[s] ] = Sum_a
            
        if ( np.linalg.norm( V_curr - V_prev ) < Eps ) :
            if ( verbose == True ) : print("n_iter_eval for convergence : ",n_iter_eval)
            break 
        
        n_iter_eval += 1 
        V_prev = V_curr

    return V_curr


In [11]:
V = Policy_eval ( env , deter_policy , Eps=1e-5 , discount=0.99 )

n_iter_eval for convergence :  108


In [12]:
V

array([-0.93690001,  0.        , -0.44592837, -0.50296922, -0.32583501,
        0.        , -0.27761546, -0.2574881 , -0.1225327 ,  0.        ,
        0.        ])

**Note** : Since state 1 and 5 are terminal states there will be no policies for them 

In [13]:
def Policy_iteration ( env , Eps=1e-5 , discount=0.99 , init='Uniform' , verbose = False  ) : 
    
    
    # init 
    statedic , MDP = env.getMDP()
    P_prev = gen_policy ( S , A , init )
    actions = np.arange( env.nA )
    
    n_iter_policy = 1
    while ( True ): 
        # Policy Evaluation 
        V = Policy_eval ( env , P_prev , Eps=Eps , discount=discount , verbose=False )
        
        # Improvement
        P_curr = np.zeros ( P_prev.shape )
        for s in MDP :
            # lookahead
            acts = np.zeros( env.nA ) 
            for a in MDP[s] :
                Sum_snext = 0
                for proba, next_state, reward, done in MDP[s][a] :
                    Sum_snext +=   (proba * ( reward + discount * V[ statedic[next_state] ] ))
                acts[a] = Sum_snext
            # select the argmax         
            P_curr[ statedic[s] ][ acts.argmax() ] = 1.0
            
        if ( np.array_equal(P_prev,P_curr) ) :
            if ( verbose ) : print("n_iter_policy for convergence : ",n_iter_policy)
            break
            
        n_iter_policy += 1 
        
        # Printing
        if ( verbose ) : 
            print("--------------------")
            print(P_prev)
            print(P_curr)
            print("--------------------")
        P_prev = P_curr

    return P_curr
    

In [14]:
P1 = Policy_iteration ( env, Eps=1e-5 , discount=0.99   )

In [15]:
P1

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [16]:
def Value_iteration ( env , Eps=1e-5 , discount=0.99 , init='Uniform' , verbose = True  ) :
    
    # init 
    statedic , MDP = env.getMDP()
    V_prev = np.random.rand( env.nS )
    actions = np.arange( env.nA )
    
    P = np.zeros  ( (env.nS,env.nA) )
    
    n_iter = 1 
    while ( True ) :
        
        # value iteration 
        V_curr = np.zeros ( V_prev.shape ) 
        # for each state s_t
        for s in MDP :
            # lookahead
            acts = np.zeros( env.nA ) 
            for a in MDP[s] :
                Sum_snext = 0
                for proba, next_state, reward, done in MDP[s][a] :
                    Sum_snext +=   (proba * ( reward + discount * V_prev[ statedic[next_state] ] ))
                acts[a] = Sum_snext
                    
            V_curr[ statedic[s] ] = acts.max()
            
        if ( np.linalg.norm( V_curr - V_prev ) < Eps ) :
            if ( verbose == True ) : print("n_iter_value_iteration for convergence : ",n_iter)
            break
            
        n_iter += 1
        V_prev = V_curr
            
    V  = V_curr
    # create a policy
    for s in MDP :
        # lookahead
        acts = np.zeros( env.nA ) 
        for a in MDP[s] :
            Sum_snext = 0
            for proba, next_state, reward, done in MDP[s][a] :
                Sum_snext +=   (proba * ( reward + discount * V[ statedic[next_state] ] ))
            acts[a] = Sum_snext
        # select the argmax         
        P[ statedic[s] ][ acts.argmax() ] = 1.0
    
    return P 

In [17]:
P2 = Value_iteration ( env, Eps=1e-5 , discount=0.99   )

n_iter_value_iteration for convergence :  85


In [18]:
P2

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [19]:
print ( "Converging to the same Determinstic policy  Confirmation = " , np.array_equal ( P1 , P2 ) ) 

Converging to the same Determinstic policy  Confirmation =  True


## Agents

In [20]:
Method = "Policy_iteration"

In [21]:
class RandomAgent(object):
    """The world's simplest agent!"""

    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done):
        return self.action_space.sample()
    
class DeterministicPolicy_Agent(object):
    """An agent using a deterministic policy generated by Policy Iteration Algorithm !"""
    
    def __init__(self, env, Method="Policy_iteration", init="Uniform", discount=0.99 ):
        
        self.action_space = env.action_space
        self.policy       = globals()[Method]( env , Eps=1e-5 , discount=discount , init=init , verbose = False  )
                        
    def act(self, observation , reward , done ):
        return self.policy[observation].argmax()
    
    
class Epsilon_Policy_Agent(object):
    """An agent using a deterministic policy generated by Policy Iteration Algorithm !"""
    def __init__(self, action_space , env , eps = 0.9 ):
        self.action_space = action_space
        self.policy       = policy
        self.eps          = eps
    
    def act(self, observation , reward , done ):
        if ( np.random.rand() <= self.eps ) :
            return self.policy[observation].argmax()
        else :
            return self.action_space.sample()

In [22]:
# Faire un fichier de log sur plusieurs scenarios
outdir = 'gridworld-v0/random-agent-results'
envm = wrappers.Monitor(env, directory=outdir, force=True )
env.setPlan("gridworldPlans/plan0.txt", {0: -0.1, 3: 1, 4: 1, 5: -1, 6: -1})
env.seed()  # Initialiser le pseudo aleatoire

# agent 
agent = DeterministicPolicy_Agent( env  )

episode_count = 1000
reward = 0
done = False
rsum = 0
FPS = 0.0001
for i in range(episode_count):
    obs = envm.reset()
    env.verbose = (i % 100 == 0 and i > 0)  # afficher 1 episode sur 100
    if env.verbose:
        env.render(FPS)
    j = 0
    rsum = 0
    while True:
        # convert state string to its encoding number 
        encoded_state = statedic[str(obs.tolist())]
        action = agent.act( encoded_state , reward , done )
        obs, reward, done, _ = envm.step(action)
        rsum += reward
        j += 1
        if env.verbose:
            env.render(FPS)
        if done:
            print("Episode : " + str(i) + " rsum=" + str(rsum) + ", " + str(j) + " actions")
            break

print("done")
env.close()

DependencyNotInstalled: Found neither the ffmpeg nor avconv executables. On OS X, you can install ffmpeg via `brew install ffmpeg`. On most Ubuntu variants, `sudo apt-get install ffmpeg` should do it. On Ubuntu 14.04, however, you'll need to install avconv with `sudo apt-get install libav-tools`.

In [ ]:
!install ffmpeg